In [1]:
import numpy as np
from tqdm import tqdm
import random

In [30]:
dataset = np.load("../data/formatted/dataset.npy", allow_pickle=True)
midi_dataset = np.load("../data/formatted/midi_dataset.npy", allow_pickle=True)
meta_dataset = np.load("../data/formatted/meta_augmented.npy", allow_pickle=True)

print(dataset.shape, midi_dataset.shape, meta_dataset.shape)

(48072, 2048) (48072, 2048, 8) (48072,)


In [31]:
print(dataset[0])

['Medium Swing' 'Tonality' 'C major' ... '<pad>' '<pad>' '<pad>']


In [34]:
#Token from dataset

chords_array = [[item for item in row] for row in dataset]
concatenated_array = np.hstack(chords_array) 
tokens = set(concatenated_array)

print(len(tokens))
print(tokens)
np.save("../data/formatted/tokens.npy", list(tokens))

264
{'A minor', '1.5989583333333333', 'Rock Waltz', 'D major', 'Beatles', 'Form_C', 'Gb major', '0.7994791666666666', 'Power Ballad', 'Db minor', 'add #7', 'E##', '2.0', 'Medium Funk', 'Slow Bossa', '0.5703125', 'C##', 'C major', 'E minor', 'maj6', '0.3997395833333333', 'Fast Swing', 'o_maj7', 'alter #5', '1.1419270833333333', 'Pop', '/', 'Blues', 'A major', 'Electro Pop', 'Funk Rock', 'Fusion', 'March', 'Foxtrot', 'F#', ':|', 'D#', 'G minor', 'Afro-Samba', 'aug', 'Fast Blues', 'E', 'F minor', 'E#', 'Medium Pop', 'Medium Swing', 'alter b5', 'Pop Shuffle', 'C', 'Rock Slow', 'add #11', 'Even 16ths', 'add b13', '0.5', 'Medium Rock', 'Shuffle Blues', 'o7', 'add b9', 'Medium Up', 'Pop-Shuffle', 'Waltz', 'Bolero', 'Samba-Funk', 'G##', 'Repeat_1', 'Blues Rock', 'Blues Shuffle', 'Bossa Acoustic', 'Shuffle', 'Bbb', 'Repeat_2', 'Reggae', 'Eb major', '0.8880208333333334', 'Ab major', 'C#', 'Maxixe', 'RnB', 'Slowly', 'major-13th', 'Medium Slow', 'Medium Up Swing', 'maj7', '2.6666666666666665', '<e

In [35]:
stoi = { tk:i for i,tk in enumerate(tokens) }
itos = { i:tk for i,tk in enumerate(tokens) }

print(stoi)

{'A minor': 0, '1.5989583333333333': 1, 'Rock Waltz': 2, 'D major': 3, 'Beatles': 4, 'Form_C': 5, 'Gb major': 6, '0.7994791666666666': 7, 'Power Ballad': 8, 'Db minor': 9, 'add #7': 10, 'E##': 11, '2.0': 12, 'Medium Funk': 13, 'Slow Bossa': 14, '0.5703125': 15, 'C##': 16, 'C major': 17, 'E minor': 18, 'maj6': 19, '0.3997395833333333': 20, 'Fast Swing': 21, 'o_maj7': 22, 'alter #5': 23, '1.1419270833333333': 24, 'Pop': 25, '/': 26, 'Blues': 27, 'A major': 28, 'Electro Pop': 29, 'Funk Rock': 30, 'Fusion': 31, 'March': 32, 'Foxtrot': 33, 'F#': 34, ':|': 35, 'D#': 36, 'G minor': 37, 'Afro-Samba': 38, 'aug': 39, 'Fast Blues': 40, 'E': 41, 'F minor': 42, 'E#': 43, 'Medium Pop': 44, 'Medium Swing': 45, 'alter b5': 46, 'Pop Shuffle': 47, 'C': 48, 'Rock Slow': 49, 'add #11': 50, 'Even 16ths': 51, 'add b13': 52, '0.5': 53, 'Medium Rock': 54, 'Shuffle Blues': 55, 'o7': 56, 'add b9': 57, 'Medium Up': 58, 'Pop-Shuffle': 59, 'Waltz': 60, 'Bolero': 61, 'Samba-Funk': 62, 'G##': 63, 'Repeat_1': 64, 'Bl

In [36]:
#Get the number of real songs
realSongs = len(dataset)/12
tenPercent = int(0.1 * realSongs)

#random a number without repeating number
randomList = random.sample(range(0, int(realSongs)), tenPercent)

#if number is bigger than 12 multiply it by 12
for i in range(len(randomList)):
    randomList[i] = randomList[i] * 12

#populate a random list with the 12 subsequent numbers per value
final_random_list=[]
for number in randomList:
    for i in range(12):
        final_random_list.append(number+i)

#check if a number is duplicated 
print(len(final_random_list) == len(set(final_random_list)))

True


In [37]:
#save the random list
np.save('../data/formatted/final_random_list.npy', final_random_list)

In [38]:
#split dataset and validation using the random list
dataset_test = dataset[final_random_list]
midi_test = midi_dataset[final_random_list]
meta_test = meta_dataset[final_random_list]

dataset_train = np.delete(dataset, final_random_list, axis=0)
midi_train = np.delete(midi_dataset, final_random_list, axis=0)

In [39]:
#save the train and test dataset
#test
np.save('../data/formatted/dataset_test.npy', dataset_test)
np.save('../data/formatted/midi_test.npy', midi_test)
np.save('../data/formatted/meta_test.npy', meta_test)
#train
np.save('../data/formatted/dataset_train.npy', dataset_train)
np.save('../data/formatted/midi_train.npy', midi_train)

In [40]:
import random
#create a file with shuffled reference index
def createWindowedShuffleReference(type, size, window, save = False):
    s = np.arange(0, size, 1)
    #num = np.arange(0, len(data)/10, 1)
    np.random.shuffle(s)

    n = int(size/window)
    numlist = random.sample(range(n), n)
    numlist = np.array(numlist)
    numlist = numlist * window

    m = np.max(numlist)
    l_ref = size-window
    print('real:', size, 'max:', m, 'length_ref:',l_ref)

    if m != l_ref:
        rest = m - l_ref
        numlist = numlist - rest

    ref = []
    for num in numlist:
        if num == 0:
            print("OK")
        for i in range(0,window):
            ref.append(num+i)

    #return the shuffled list
    if save:
        np.savetxt("../data/shuffle_" + type + ".txt", ref, fmt='%i', delimiter=" ", header='Array shape: ('+str(size)+', 1)')
    return ref

In [41]:
def getData(folder, name):
    data_path = folder + '/' + name
    data = np.loadtxt(data_path)
    f = open(data_path, "r")
    format = f.readline().replace('# Array shape: (', '').replace('\n', '').replace(')', '')
    format = np.array(format.split(', ')).astype(int)
    f.close()
    return data, format

In [42]:
train_dataset = np.load('../data/formatted/dataset_train.npy', allow_pickle=True)
test_dataset = np.load('../data/formatted/dataset_test.npy', allow_pickle=True)

train_midi = np.load('../data/formatted/midi_train.npy', allow_pickle=True)
test_midi = np.load('../data/formatted/midi_test.npy', allow_pickle=True)

print(train_dataset.shape, train_midi.shape, test_dataset.shape, test_midi.shape)

BATCH_SHUFFLE_SIZE = 1
ref = createWindowedShuffleReference("train", len(train_dataset), BATCH_SHUFFLE_SIZE, True)
ref_test = createWindowedShuffleReference("test", len(test_dataset), BATCH_SHUFFLE_SIZE, True)

# first shuffle the train dataset
shuffle_train, format_train = getData('../data', 'shuffle_train.txt')
shuffle_train = shuffle_train.reshape(format_train[0], ).astype(int)
shuffle_train = shuffle_train.tolist()
dataset = train_dataset[shuffle_train]
midiDataset = train_midi[shuffle_train]

#second shuffle the test dataset
shuffle_test, format_test = getData('../data', 'shuffle_test.txt')
shuffle_test = shuffle_test.reshape(format_test[0], ).astype(int)
shuffle_test = shuffle_test.tolist()
validation = test_dataset[shuffle_test]
midi_validation = test_midi[shuffle_test]

(43272, 2048) (43272, 2048, 8) (4800, 2048) (4800, 2048, 8)
real: 43272 max: 43271 length_ref: 43271
OK
real: 4800 max: 4799 length_ref: 4799
OK


In [43]:
np.save('../data/shuffled/dataset_train.npy', dataset)
np.save('../data/shuffled/midi_train.npy', midiDataset)
np.save('../data/shuffled/dataset_test.npy', validation)
np.save('../data/shuffled/midi_test.npy', midi_validation)

Checkpoint

In [4]:
%reload_ext autoreload
%autoreload 2
from utils import *

max_length = 2048
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [5]:
print(train.shape, test.shape, midi_train.shape, midi_test.shape)

dataset = TokenDatasetMidi(train, midi_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_test, max_length, tokens)

(43272, 2048) (4800, 2048) (43272, 2048, 8) (4800, 2048, 8)
data has 43272 pieces, 264 unique tokens.
data has 4800 pieces, 264 unique tokens.


In [6]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 250,
    }
)

03/11/2024 19:13:25 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


In [2]:
%reload_ext autoreload
%autoreload 2
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
print("Available devices: ", torch.cuda.device_count())

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

Available devices:  2


In [3]:
print("Available devices: ", torch.cuda.device_count())
print("torch version:", torch.__version__)
print("cudnn version:", torch.backends.cudnn.version())
print("cuda version:", torch.version.cuda)

Available devices:  2
torch version: 2.2.1+cu121
cudnn version: 8902
cuda version: 12.1


In [8]:
epochs = 250
embedding = 192
heads = 4
layers = 4
batch_size = 64
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()

03/11/2024 19:14:03 - INFO - model -   number of parameters: 1.887872e+06
03/11/2024 19:14:03 - INFO - model -   number of parameters: 1.887872e+06


../models/model_epochs->250_heads->4_embd->192_batch->64_new_midi_embeddings


  0%|          | 0/677 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/workspace/src/utils.py", line 101, in __getitem__
    dix = [self.stoi[s] for s in chunk[0]]
  File "/workspace/src/utils.py", line 101, in <listcomp>
    dix = [self.stoi[s] for s in chunk[0]]
KeyError: 4.0
